In [ ]:
import nltk
import string
import re 
import itertools
import heapq
import operator
import networkx as nx
import copy
import networkx as nx
from nltk.corpus import stopwords
from nltk import pos_tag
import matplotlib.pyplot as plt
import os
from gensim.models.keyedvectors import KeyedVectors
from sklearn.metrics.pairwise import euclidean_distances as ed
import numpy as np
import os
import pickle
from operator import mul
from nltk.metrics import scores
word_vectors = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz', binary=True)

## Text Clean and Tokenisation

In [ ]:
def clean_text_simple(text, remove_stopwords=True, pos_filtering=True, stemming=True):
    
    punct = string.punctuation.replace('-', '')
    # remove punctuation (preserving intra-word dashes)
    text = ''.join(l for l in text if l not in punct)
    # strip extra white space
    text = re.sub(' +',' ',text)
    # strip leading and trailing white space
    text = text.strip()
    # tokenize (split based on whitespace)
    tokens = text.split(' ')
    if pos_filtering == True:
        # apply POS-tagging
        tagged_tokens = pos_tag(tokens)
        # retain only nouns and adjectives
        tokens_keep = []
        for i in range(len(tagged_tokens)):
            item = tagged_tokens[i]
            if (
            item[1] == 'NN' or
            item[1] == 'NNS' or
            item[1] == 'NNP' or
            item[1] == 'NNPS' or
            item[1] == 'JJ' or
            item[1] == 'JJS' or
            item[1] == 'JJR'
            ):
                tokens_keep.append(item[0])
        tokens = tokens_keep
    if remove_stopwords:
        stpwds = stopwords.words('english')
        # remove stopwords
        tokens = [token for token in tokens if token not in stpwds]

    if stemming:
        stemmer = nltk.stem.PorterStemmer()
        # apply Porter's stemmer
        tokens_stemmed = list()
        tokens_unstemmed = list()
        for token in tokens:
            tokens_stemmed.append(stemmer.stem(token).lower())
            tokens_unstemmed.append(token)

    return(tokens_stemmed,tokens_unstemmed)

## Build Simple Graph

In [ ]:
def terms_to_graph(terms, w):
    # This function returns a directed, weighted networkx graph from a list of terms (the tokens from the pre-processed text) e.g., ['quick','brown','fox']
    # Edges are weighted based on term co-occurence within a sliding window of fixed size 'w'
    
    from_to = {}
    
    # create initial complete graph (first w terms)
    w = min(w,len(terms))
    terms_temp = terms[0:w]
    indexes = list(itertools.combinations(range(w), r=2))
    
    new_edges = []
    
    for my_tuple in indexes:
        new_edges.append(tuple([terms_temp[i] for i in my_tuple]))
    
    for new_edge in new_edges:
        if new_edge in from_to:
            from_to[new_edge] += 1
        else:
            from_to[new_edge] = 1
    
    # then iterate over the remaining terms
    for i in xrange(w, len(terms)):
        # term to consider
        considered_term = terms[i]
        # all terms within sliding window
        terms_temp = terms[(i-w+1):(i+1)]
        
        # edges to try
        candidate_edges = []
        for p in xrange(w-1):
            candidate_edges.append((terms_temp[p],considered_term))
            
        for try_edge in candidate_edges:
        
            # if not self-edge
            if try_edge[1] != try_edge[0]:
                
                # if edge has already been seen, update its weight
                if try_edge in from_to:
                    from_to[try_edge] += 1
                
                # if edge has never been seen, create it and assign it a unit weight     
                else:
                    from_to[try_edge] = 1
    
    # create empty graph
    g = nx.DiGraph()
    
    for edge in from_to :
        g.add_edge(edge[0],edge[1],weight=from_to[edge])
    
    degree_dict = g.degree(weight='weight')
    nx.set_node_attributes(g,'weight',degree_dict)
    return(g)

## Word Attraction Force Helper Functions

In [ ]:
def my_vector_getter(word, word_vectors):
    try:
        word_representation = word_vectors[word].reshape(1,-1)
        return (word_representation)
    except KeyError:
        return (np.random.uniform(-0.25,0.25,300).reshape(1,-1))
        
def my_euclidean_distance(word1, word2, word_vectors):
    distance = ed(my_vector_getter(word1, word_vectors),my_vector_getter(word2, word_vectors))
    return (round(distance, 4))

## WAF Graph Builder

In [ ]:
def terms_to_graph_word_attraction(terms_stemmed,terms_unstemmed,w):
    # This function returns a directed, weighted networkx graph from a list of terms (the tokens from the pre-processed text) e.g., ['quick','brown','fox']
    # Edges are weighted based on term co-occurence, word2vec vector and the respective frequencies within a sliding window of fixed size 'w'
    from_to={}
    
    # create initial complete graph (first w terms)
    w = min(w,len(terms_stemmed))
    terms_temp = terms_stemmed[0:w]
    indexes = list(itertools.combinations(range(w), r=2))
    
    new_edges = []
    

    for my_tuple in indexes:
        new_edges.append(tuple([terms_temp[i] for i in my_tuple]))
    
    for new_edge in new_edges:
        if new_edge in from_to:
            from_to[new_edge] += 1
        else:
            from_to[new_edge] = 1
    
    # then iterate over the remaining terms
    for i in xrange(w, len(terms_stemmed)):
        # term to consider
        considered_term = terms_stemmed[i]
        # all terms within sliding window
        terms_temp = terms_stemmed[(i-w+1):(i+1)]
        
        # edges to try
        candidate_edges = []
        for p in xrange(w-1):
            candidate_edges.append((terms_temp[p],considered_term))
            
        for try_edge in candidate_edges:
        
            # if not self-edge
            if try_edge[1] != try_edge[0]:
                
                # if edge has already been seen, update its weight
                if try_edge in from_to:
                    from_to[try_edge] += 1
                
                # if edge has never been seen, create it and assign it a unit weight     
                else:
                    from_to[try_edge] = 1
    
    # create empty graph
    min_attr = float("inf")
    edgelist = from_to.keys()
    waf = {}
    for edge in edgelist :
        word1 = edge[0]
        word2 = edge[1]
        if word1 != word2 :
            word1_freq = terms_stemmed.count(word1)
            word2_freq = terms_stemmed.count(word2)
            word1_unstemmed = terms_unstemmed[terms_stemmed.index(word1)]
            word2_unstemmed = terms_unstemmed[terms_stemmed.index(word2)]
            distance = my_euclidean_distance(word1_unstemmed,word2_unstemmed,word_vectors)
            force = round(word1_freq * word2_freq / float(distance * distance), 5)
            dice = 2*from_to[edge]/(word1_freq*word2_freq)
            attr = dice*force
            if attr!=0:
                min_attr = min(attr,min_attr)
                waf[edge] = attr
    
    g = nx.DiGraph()
    for item in waf :
        if waf[item]!=0:
            waf[item] = round(waf[item]*1.0/min_attr)
            g.add_edge(item[0],item[1],weight=waf[item])
    
    degree_dict = g.degree(weight='weight')
    nx.set_node_attributes(g,'weight',degree_dict)
    return(g)

## Core Decomposition

In [ ]:
def core_dec(g, weighted = True):
    # unweighted or weighted k-core decomposition of a graph
  
    # work on clone of g to preserve g 
    gg = copy.deepcopy(g)    
    
    # initialize dictionary that will contain the core numbers
    cores_g = dict((key,0) for key in gg.nodes())
    
    if weighted == True:
        # k-core decomposition for weighted graphs (generalized k-cores)
        # based on Batagelj and Zaversnik's (2002) algorithm #4
    
        # initialize min heap of degrees
        degree_dict = gg.degree(weight='weight')
        heap_g = [[degree_dict[node],node] for node in degree_dict]
        heapq.heapify(heap_g)
        while len(heap_g)>0:
            
            top = heap_g[0][1]
            # save names of its neighbors
            neighbors_top = gg[top].keys()
            # set core number of heap top element as its weighted degree
            cores_g[top] = gg.node[top]['weight']
            
            # delete top vertice (weighted degrees are automatically updated)
            gg.remove_node(top)
            
            if len(neighbors_top)>0:
            # iterate over neighbors of top element
                for i, name_n in enumerate(neighbors_top):
                    max_n = max(cores_g[top],gg.degree(weight='weight')[name_n])
                    gg.node[name_n]["weight"] = max_n
                    # update heap
                    heap_g = [[gg.node[nodex]['weight'],nodex] for nodex in gg.nodes()]
                    heapq.heapify(heap_g)
            else:
                # update heap
                heap_g = [[gg.node[nodex]['weight'],nodex] for nodex in gg.nodes()]
                heapq.heapify(heap_g)
                
    else:
        # k-core decomposition for unweighted graphs
        # based on Batagelj and Zaversnik's (2002) algorithm #1
        cores_g = nx.core_number(gg)
    
    # sort vertices by decreasing core number
#     sorted_cores_g = sorted(cores_g.items(), key=operator.itemgetter(1), reverse=True)
    
    return cores_g

## Weighted K-Peak Decomposition

In [ ]:
def get_kpeak_decomposition(G,weighted = True):
    if weighted == True :
        orig_core_nums = core_dec(G,weighted)
        current_core_nums = dict(orig_core_nums)
    else :
        orig_core_nums = nx.core_number(G)
        current_core_nums = orig_core_nums.copy()
        
    H = G.copy(); 
    H_nodes = set(G.nodes())
    
    
    peak_numbers = {}

    # Each iteration of the while loop finds a k-contour
    if weighted == True :
        while(len(H.nodes()) > 0):
            temp_dict = dict(current_core_nums)
            sorted_tracker = sorted(current_core_nums.items(), key=operator.itemgetter(1), reverse=True)
            degen_number = sorted_tracker[0][1]
            kcontour_nodes = []
            for x in current_core_nums :
                if current_core_nums[x] == degen_number :
                    kcontour_nodes.append(x)
            for n in kcontour_nodes :
                peak_numbers[n] = temp_dict[n]
            # Removing the kcontour (i.e. degeneracy) and re-computing core numbers.
            H_nodes = H_nodes.difference(set(kcontour_nodes))
            H = G.subgraph(list(H_nodes))
            current_core_nums = core_dec(H)
    
    else :
        while(len(H.nodes()) > 0):
            
            # degen_core is the degeneracy of the graph
            degen_core = nx.k_core(H) # Degen-core

            # Nodes in the k-contour. Their current core number is their peak number.
            kcontour_nodes = degen_core.nodes()
            for n in kcontour_nodes:
                peak_numbers[n] = current_core_nums[n]

            # Removing the kcontour (i.e. degeneracy) and re-computing core numbers.
            H_nodes = H_nodes.difference(set(kcontour_nodes))
            H = G.subgraph(list(H_nodes))
            current_core_nums = nx.core_number(H)
    
    # sort vertices by decreasing peak number
#     sorted_peaks_g = sorted(peak_numbers.items(), key=operator.itemgetter(1), reverse=True)
    return peak_numbers

## Rank Computation

In [ ]:
def compute_rank(core_numbers, g):
    core_rank = {}
    for node in core_numbers :
        core_rank[node] = sum([core_numbers[neighbor] for neighbor in g[node].keys()])
        #core_rank[node] = reduce(mul,[core_numbers[neighbor] for neighbor in g[node].keys()])
        #core_rank[node] = sum([pow(core_numbers[neighbor],g[node][neighbor]['weight']) for neighbor in g[node].keys()])
    sorted_ranks = sorted(core_rank.items(), key=operator.itemgetter(1), reverse=True)
    sorted_ranks = [x[0] for x in sorted_ranks]
    return sorted_ranks

## Hulth Dataset

In [ ]:
total = os.listdir("EMNLP_2016-master/data/Hulth2003/validation_training/validation/")
path = "EMNLP_2016-master/data/Hulth2003/validation_training/validation/"
abstract_files = [filename for filename in total if '.abstr' in filename]
stemmer = nltk.stem.PorterStemmer()
crp_normal_p,crp_waf_p,pr_unweighted_p,pr_normal_p,pr_waf_p,pn_p,pn_u_p,pn_waf_p = 0,0,0,0,0,0,0,0
crp_normal_r,crp_waf_r,pr_unweighted_r,pr_normal_r,pr_waf_r,pn_r,pn_u_r,pn_waf_r = 0,0,0,0,0,0,0,0
crp_normal_f,crp_waf_f,pr_unweighted_f,pr_normal_f,pr_waf_f,pn_f,pn_u_f,pn_waf_f = 0,0,0,0,0,0,0,0

crp,crp_waf,prp_u,prp,prp_waf,pnp,pnp_u,pnp_waf = [],[],[],[],[],[],[],[]

for window_size in range(3,15) :
    print "Window Size = ",window_size
    crp_normal_p,crp_waf_p,pr_unweighted_p,pr_normal_p,pr_waf_p,pn_p,pn_u_p,pn_waf_p = 0,0,0,0,0,0,0,0
    crp_normal_r,crp_waf_r,pr_unweighted_r,pr_normal_r,pr_waf_r,pn_r,pn_u_r,pn_waf_r = 0,0,0,0,0,0,0,0
    crp_normal_f,crp_waf_f,pr_unweighted_f,pr_normal_f,pr_waf_f,pn_f,pn_u_f,pn_waf_f = 0,0,0,0,0,0,0,0
    for filename in abstract_files :
        filepath =  path+filename
        with open(filepath) as file :
            text = file.read()
        text = " ".join(text.strip().split())
        tokens_stemmed,tokens_unstemmed = clean_text_simple(text)
        #Traditional Graph of Words
        G = terms_to_graph(tokens_stemmed,window_size)
    #     H = nx.convert_node_labels_to_integers(G)
    #     nx.write_edgelist(H,'edgelist.txt',comments='#', delimiter=' ', data=False, encoding='utf-8')

        G.remove_edges_from(G.selfloop_edges())
        core_number_normal_weighted =  core_dec(G)
        peak_number_normal_unweighted = get_kpeak_decomposition(G,False)
        peak_number_normal_weighted = get_kpeak_decomposition(G)
        #Word Attraction Force Graph of Words
        G_waf = terms_to_graph_word_attraction(tokens_stemmed,tokens_unstemmed,window_size)
        G_waf.remove_edges_from(G_waf.selfloop_edges())
        core_number_waf_weighted =  core_dec(G_waf)
        peak_number_waf_weighted = get_kpeak_decomposition(G_waf)


        core_rank_normal = compute_rank(core_number_normal_weighted,G)
        peak_rank_normal = compute_rank(peak_number_normal_weighted,G)
        peak_rank_unweighted = compute_rank(peak_number_normal_unweighted,G)
        
        
        sorted_pn_weighted = sorted(peak_number_normal_weighted.items(), key=operator.itemgetter(1), reverse=True)
        sorted_pn_weighted = [x[0] for x in sorted_pn_weighted]
        sorted_pn_unweighted = sorted(peak_number_waf_weighted.items(), key=operator.itemgetter(1), reverse=True)
        sorted_pn_unweighted = [x[0] for x in sorted_pn_unweighted]
        sorted_pn_waf = sorted(peak_number_normal_unweighted.items(), key=operator.itemgetter(1), reverse=True)
        sorted_pn_waf = [x[0] for x in sorted_pn_waf]
    
        core_rank_waf = compute_rank(core_number_waf_weighted,G_waf)
        peak_rank_waf = compute_rank(peak_number_waf_weighted,G_waf)


        cr_n = set(core_rank_normal[:int(len(core_rank_normal)/3)])
        cr_waf = set(core_rank_waf[:int(len(core_rank_waf)/3)])
        pr_u = set(peak_rank_unweighted[:int(len(peak_number_normal_unweighted)/3)])
        pr_n = set(peak_rank_normal[:int(len(core_rank_normal)/3)])
        pr_waf = set(peak_rank_waf[:int(len(core_rank_waf)/3)])
        
        pn_w = set(sorted_pn_weighted[:int(len(sorted_pn_weighted)/3)])
        pn_u = set(sorted_pn_weighted[:int(len(sorted_pn_unweighted)/3)])
        pn_waf = set(sorted_pn_waf[:int(len(sorted_pn_waf)/3)])
        
        keywordspath = filepath.replace('abstr','uncontr')
        with open(keywordspath) as file :
            keywords = file.read()
        keywords =  " ".join(keywords.strip().split()).split(';')
        keywords = [key.strip().split() for key in keywords]
        keywords = set([stemmer.stem(word).lower() for key in keywords for word in key])
        
        crp_normal_p+=scores.precision(keywords,cr_n)
        crp_normal_r+=scores.recall(keywords,cr_n)
        crp_normal_f+=scores.f_measure(keywords,cr_n)

        crp_waf_p+=scores.precision(keywords,cr_waf)
        crp_waf_r+=scores.recall(keywords,cr_waf)
        crp_waf_f+=scores.f_measure(keywords,cr_waf)
        
        pr_normal_p+=scores.precision(keywords,pr_n)
        pr_normal_r+=scores.recall(keywords,pr_n)
        pr_normal_f+=scores.f_measure(keywords,pr_n)

        pr_waf_p+=scores.precision(keywords,pr_waf)
        pr_waf_r+=scores.recall(keywords,pr_waf)
        pr_waf_f+=scores.f_measure(keywords,pr_waf)
        
        pr_unweighted_p+=scores.precision(keywords,pr_u)
        pr_unweighted_r+=scores.recall(keywords,pr_u)
        pr_unweighted_f+=scores.f_measure(keywords,pr_u)

        pn_p+=scores.precision(keywords,pn_w)
        pn_r+=scores.recall(keywords,pn_w)
        pn_f+=scores.f_measure(keywords,pn_w)


        pn_u_p+=scores.precision(keywords,pn_u)
        pn_u_r+=scores.recall(keywords,pn_u)
        pn_u_f+=scores.f_measure(keywords,pn_u)

        pn_waf_p+=scores.precision(keywords,pn_waf)
        pn_waf_r+=scores.recall(keywords,pn_waf)
        pn_waf_f+=scores.f_measure(keywords,pn_waf)
                                
    crp.append([crp_normal_f,crp_normal_p,crp_normal_r])
    crp_waf.append([crp_waf_f,crp_waf_p,crp_waf_r])
    prp.append([pr_normal_f,pr_normal_p,pr_normal_r])
    prp_waf.append([pr_waf_f,pr_waf_p,pr_waf_r])
    prp_u.append([pr_unweighted_f,pr_unweighted_p,pr_unweighted_r])
    pnp.append([pn_f,pn_p,pn_r])
    pnp_u.append([pn_u_f,pn_u_p,pn_u_r])
    pnp_waf.append([pn_waf_f,pn_waf_p,pn_waf_f])

In [ ]:
allp = [crp,crp_waf,prp_u,prp,prp_waf,pnp,pnp_u,pnp_waf]
for x in allp :
    a = sorted(x,reverse=True)[0]
    print
    for y in a :
        print y/5,